In [118]:
import pandas as pd
import numpy as np

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# for imputation
from pyod.models.knn  import KNN
import pyod

#model building
from sklearn.model_selection import train_test_split,KFold,cross_val_score,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
from lightgbm import LGBMClassifier


pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [42]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.columns = [col.lower() for col in train.columns]
test.columns  = [col.lower() for col in test.columns]

train.index = train.loan_id
train.drop(["loan_id"],axis=1,inplace = True)
train["loan_status"].replace({"Y":1,"N":0},inplace=True)

test.index = test.loan_id
test.drop(["loan_id"],axis=1,inplace = True)

In [43]:
train.head()

,gender,married,dependents,education,self_employed,applicantincome,coapplicantincome,loanamount,loan_amount_term,credit_history,property_area,loan_status
loan_id,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [44]:
test.head()

,gender,married,dependents,education,self_employed,applicantincome,coapplicantincome,loanamount,loan_amount_term,credit_history,property_area
loan_id,,,,,,,,,,,
LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [145]:
round(train.describe(),2).T

,count,mean,std,min,25%,50%,75%,max
applicantincome,614.0,5403.46,6109.04,150.00,2877.50,3812.50,5795.00,81000.00
coapplicantincome,614.0,1621.25,2926.25,0.00,0.00,1188.50,2297.25,41667.00
loanamount,592.0,146.41,85.59,9.00,100.00,128.00,168.00,700.00
loan_amount_term,600.0,342.00,65.12,12.00,360.00,360.00,360.00,480.00
credit_history,564.0,0.84,0.36,0.00,1.00,1.00,1.00,1.00
loan_status,614.0,0.69,0.46,0.00,0.00,1.00,1.00,1.00
loanbyincome,592.0,11.85,21.66,0.00,0.00,8.93,17.81,375.97
loanbyterm,578.0,0.48,0.53,0.02,0.29,0.37,0.51,9.25


In [146]:
round(test.describe(),2).T

,count,mean,std,min,25%,50%,75%,max
applicantincome,367.0,4805.60,4910.69,0.0,2864.00,3786.00,5060.00,72529.00
coapplicantincome,367.0,1569.58,2334.23,0.0,0.00,1025.00,2430.50,24000.00
loanamount,362.0,136.13,61.37,28.0,100.25,125.00,158.00,550.00
loan_amount_term,361.0,342.54,65.16,6.0,360.00,360.00,360.00,480.00
credit_history,338.0,0.83,0.38,0.0,1.00,1.00,1.00,1.00
loanbyincome,362.0,12.33,19.21,0.0,0.00,7.65,17.88,162.16
loanbyterm,356.0,0.52,1.42,0.1,0.29,0.36,0.49,21.67


In [45]:
k = pd.DataFrame()
k["train"] = train.dtypes
k['test'] = test.dtypes
k.T

,gender,married,dependents,education,self_employed,applicantincome,coapplicantincome,loanamount,loan_amount_term,credit_history,property_area,loan_status
train,object,object,object,object,object,int64,float64,float64,float64,float64,object,int64
test,object,object,object,object,object,int64,int64,float64,float64,float64,object,NaN


In [46]:
k = pd.DataFrame()
k["train"] = round(100*train.isna().sum()/len(train),2)
k["test"] = round(100*test.isna().sum()/len(test),2)
k.T

,gender,married,dependents,education,self_employed,applicantincome,coapplicantincome,loanamount,loan_amount_term,credit_history,property_area,loan_status
train,2.12,0.49,2.44,0.0,5.21,0.0,0.0,3.58,2.28,8.14,0.0,0.0
test,3.00,0.00,2.72,0.0,6.27,0.0,0.0,1.36,1.63,7.90,0.0,NaN


In [96]:
cols =[col for col in train.columns if train[col].dtype=='object']
for col in cols:
    df = train.groupby([col,'loan_status']).size().unstack()
    df["approved_ratio"] =100*round(df[1]/df.sum(axis=1),2)
    print(df,end="\n\n\n")

loan_status    0    1  approved_ratio
gender                               
Female        37   75            67.0
Male         150  339            69.0


loan_status    0    1  approved_ratio
married                              
No            79  134            63.0
Yes          113  285            72.0


loan_status    0    1  approved_ratio
dependents                           
0            107  238            69.0
1             36   66            65.0
2             25   76            75.0
3+            18   33            65.0


loan_status     0    1  approved_ratio
education                             
Graduate      140  340            71.0
Not Graduate   52   82            61.0


loan_status      0    1  approved_ratio
self_employed                          
No             157  343            69.0
Yes             26   56            68.0


loan_status     0    1  approved_ratio
property_area                         
Rural          69  110            61.0
Semiurban      54  179   

In [139]:
df = pd.DataFrame()
cols= ["applicantincome","coapplicantincome","loanamount","loanbyincome","loanbyterm"]
for col in cols:
    temp = pd.DataFrame(train.groupby(["loan_status"])[col].describe())
    temp["column"] = col
    df = df.append(round(temp,2)) 
df

,count,mean,std,min,25%,50%,75%,max,column
loan_status,,,,,,,,,
0,192.0,5446.08,6819.56,150.00,2885.00,3833.50,5861.25,81000.00,applicantincome
1,422.0,5384.07,5765.44,210.00,2877.50,3812.50,5771.50,63337.00,applicantincome
0,192.0,1877.81,4384.06,0.00,0.00,268.00,2273.75,41667.00,coapplicantincome
1,422.0,1504.52,1924.75,0.00,0.00,1239.50,2297.25,20000.00,coapplicantincome
0,181.0,151.22,85.86,9.00,100.00,129.00,176.00,570.00,loanamount
1,411.0,144.29,85.48,17.00,100.00,126.00,161.00,700.00,loanamount
0,181.0,12.38,31.60,0.00,0.00,6.91,16.66,375.97,loanbyincome
1,411.0,11.61,15.42,0.00,0.00,9.49,18.25,194.17,loanbyincome
0,175.0,0.49,0.44,0.02,0.29,0.38,0.54,4.31,loanbyterm


In [148]:
log_train = train.copy()
log_test = test.copy()
cols = ['gender', 'married','education', 'self_employed','property_area']
for col in cols:
    log_train[col].fillna('Missing',inplace=True)
log_train.fillna(log_train.mean(),inplace=True)
log_train[cols] = log_train[cols].astype(object)

log_train[cols] = log_train[cols].apply(le.fit_transform)

log_train['dependents'].replace("3+",4,inplace=True)
log_train['dependents'].fillna("5",inplace=True)
log_train['dependents'] = log_train['dependents'].astype(np.float)

clf = LogisticRegression()
X = log_train[[col for col in log_train.columns if col != "loan_status"]]
y = log_train[["loan_status"]]

#*****************************************************************************************
# model = clf.fit(X,y)
# pred = model.predict(X)
# accuracy = accuracy_score(y,pred)
# print(accuracy)
#*****************************************************************************************


#*****************************************************************************************
#cross validation
# scores = cross_val_score(clf,X,y,cv=5,scoring='roc_auc')
# print(scores)
#*****************************************************************************************

skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=101)
cv_score = []
pred_test_full =0
for train_index,test_index in skf.split(X,y):
    x_train,x_val = X.iloc[train_index],X.iloc[test_index]
    y_train,y_val = y.iloc[train_index],y.iloc[test_index]
    clf = LogisticRegression()
    clf.fit(x_train,y_train)
    score = round(roc_auc_score(y_val,clf.predict(x_val)),4)
    cv_score.append(score)
    print(score,end=",")
    
    #predictions
    pred_test = clf.predict_proba(X)[:,1]
    pred_test_full += pred_test

0.7,0.6884,0.646,0.7393,0.6986,0.6723,0.7105,0.7513,0.7393,0.7105,

In [147]:
lgbm_train = train.copy()
lgbm_test = test.copy()

lgbm_train["traintest"]='train'
lgbm_test["traintest"]='test'
lgbm_test["loan_status"]=np.nan
data = pd.concat([lgbm_test,lgbm_train])

#if large data is missing
cat_cols1 = ['self_employed']
for col in cat_cols1:
    data[col].fillna('Missing',inplace=True)

cat_cols2 = ['credit_history']
for col in cat_cols2:
    data[col].fillna(2,inplace=True)
    
#if small number of data is missing
cat_cols = ["married",'gender',"dependents"]
for col in cat_cols:
    data[col].fillna(data[col].mode()[0],inplace=True)

cat_cols = ['self_employed','credit_history',"married",'gender',"dependents",'education','property_area']
data[cat_cols] = data[cat_cols].astype(object)
data[cat_cols] = data[cat_cols].apply(le.fit_transform)

lgbm_train = data[data["traintest"]=="train"]
lgbm_train.drop(["traintest"],axis=1,inplace=True)
lgbm_test = data[data["traintest"]=="test"]
lgbm_test.drop(["traintest","loan_status"],axis=1,inplace=True)

X = lgbm_train.drop(["loan_status"],axis=1)
y = lgbm_train['loan_status']

cat_cols = ['self_employed','credit_history',"married",'gender',"dependents",'education','property_area']
cat_cols = [X.columns.get_loc(c) for c in cat_cols]
lgb = LGBMClassifier(boosting_type='gbdt', 
                      objective='binary',
                      num_iteration=1000,
                      num_leaves=10,
                      min_data_in_leaf=3,
                      max_depth=7,
                      learning_rate=0.01,
                      seed=0,
                      categorical_feature = cat_cols
                     )

skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=101)
cv_score = []
pred_test_full =0
for train_index,test_index in skf.split(X,y):
    x_train,x_val = X.iloc[train_index],X.iloc[test_index]
    y_train,y_val = y.iloc[train_index],y.iloc[test_index]
    clf = lgb
    clf.fit(x_train,y_train)
    score = round(roc_auc_score(y_val,clf.predict(x_val)),4)
    cv_score.append(score)
    print(score,end=",")
    
    #predictions
    pred_test = clf.predict_proba(X)[:,1]
    pred_test_full += pred_test

0.6767,0.7151,0.6604,0.6773,0.7393,0.6604,0.7105,0.7657,0.7036,0.7011,

In [50]:
train.head()

,gender,married,dependents,education,self_employed,applicantincome,coapplicantincome,loanamount,loan_amount_term,credit_history,property_area,loan_status,loanbyincome,loanbyterm
loan_id,,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,NaN,NaN
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,11.78125,0.355556
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0.00000,0.183333
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,19.65000,0.333333
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0.00000,0.391667


In [128]:
df = pd.DataFrame(train.groupby(["loan_status"])[col].describe())
df["column"] = col

,count,mean,std,min,25%,50%,75%,max,column
loan_status,,,,,,,,,
0,175.0,0.492564,0.436674,0.025000,0.288889,0.375000,0.541667,4.305556,loanbyterm
1,403.0,0.474974,0.560066,0.072222,0.283333,0.361111,0.500000,9.250000,loanbyterm


In [126]:
df.columns

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')